# Week 7 Assessment

### Configure your serial connection to your microcontroller. Use the following commands to establish a serial connection.
#### For Windows
 %serialconnect to --port=COM3 --baud=115200
#### For macOS
 %serialconnect to --port=/dev/tty.SLAB_USBtoUART --baud=115200

## Building an Interactive Memory-Based Game
### Scenario
You are tasked with developing a memory-based game using a Raspberry Pi Pico. The game has four LEDs and four buttons, each corresponding to a unique tone. The system generates a sequence of tones and LED flashes that players must memorise and replicate using the buttons. The game becomes progressively more challenging as the sequence length increases. If the player makes a mistake, the game ends, a "game-over" melody is played, and the system resets.

## Requirements
### 1.	Hardware Setup:
🅰️ Assemble and connect the following components on a breadboard:

    ➡️ Four LEDs (different colours for clarity).
   
    ➡️ Four buttons 
   
    ➡️ One passive buzzer.
   
    ➡️ Raspberry Pi Pico.
   
    ➡️ Jumper wires for connectivity

### 2.	Programming:

🅱️ Program the microcontroller using MicroPython to:

    ➡️ Randomly generate and play an increasing sequence of tones and LED flashes.
   
    ➡️ Capture user inputs through button presses.
    
    ➡️ Compare user inputs with the generated sequence and provide real-time feedback.

    ➡️ Play a "game-over" melody and reset the game if a mistake is made.


### 3.	Game Logic:
⏩ Sequence progression: Increase sequence length as the game advances.

⏩ Error detection: Reset the game when incorrect user input is detected.

### 4.	Players should also:

    ➡️ Enter their name at the start in Terminal or Console.
    
    ➡️ See real-time score updates.
    
    ➡️ Be able to play multiple times.
    
    ➡️ View the top 5 scores in a leaderboard stored in leaderboard.txt.


### Melodies:
#### LED and Button Tunes
game_tones = [196, 261, 329, 784]

#### Level up Tune
def play_level_up_tune():

    for tone in [329, 392, 659, 523, 587, 784]:
        play_tone(tone, 150)
        time.sleep(0.15)

#### Game over Tune
def game_over(score, name):

    print("\nGame over! Your score:", score)
    play_tone(622, 300)
    time.sleep(0.3)
    play_tone(587, 300)
    time.sleep(0.3)
    play_tone(554, 300)
    time.sleep(0.3)
    for _ in range(10):
        for pitch in range(-10, 11): 
            play_tone(523 + pitch, 5) 
    time.sleep(0.5)
    save_score(name, score)
    show_leaderboard()
    
#### Resources:
##### File Handling in Python
https://www.geeksforgeeks.org/file-handling-python/
https://www.w3schools.com/python/python_file_handling.asp
##### Python os Module
https://www.w3schools.com/python/module_os.asp?ref=escape.tech
https://www.geeksforgeeks.org/os-module-python-examples/
##### Python Functions
https://www.w3schools.com/python/python_functions.asp
##### Python For Loops
https://www.w3schools.com/python/python_for_loops.asp
##### Python While Loops
https://www.w3schools.com/python/python_while_loops.asp
##### Python Conditions and If statements
https://www.w3schools.com/python/python_conditions.asp
##### Input and Output in Python
https://www.geeksforgeeks.org/input-and-output-in-python/
https://www.w3schools.com/python/ref_func_input.asp
https://www.w3schools.com/python/python_variables_output.asp
##### Python Logical Operators
https://www.w3schools.com/python/gloss_python_logical_operators.asp
##### Python Random random() Method
https://www.w3schools.com/python/ref_random_random.asp
https://www.geeksforgeeks.org/python-random-module/
##### Raspberry Pi Pico 2 W
https://www.raspberrypi.com/documentation/microcontrollers/pico-series.html#pico2w-technical-specification
##### Pulse Width Modulation
https://docs.micropython.org/en/latest/esp8266/tutorial/pwm.html



In [ ]:
from machine import Pin, PWM
from time import sleep
import os
import random


# Pin 0 red button
# Pin 1 white button
# Pin 2 green button

# Pin 3 buzzer

# Pin 4 red led
# Pin 5 green led
# Pin 6 blue led

# Define buttons
buttons = [
    Pin(0, Pin.IN, Pin.PULL_UP), # Red
    Pin(1, Pin.IN, Pin.PULL_UP), # White
    Pin(2, Pin.IN, Pin.PULL_UP) # Green
]

# Define RGB light pins
lights = [
    Pin(4, Pin.OUT), # Red
    Pin(5, Pin.OUT), # Green
    Pin(6, Pin.OUT)  # Blue
]

# Define buzzer
buzzer = PWM(Pin(3)) # Define pin 3 as PWM
buzzer.freq(2000) # Set the base fequency to 2000hz
buzzer.duty_u16(0) # Turn the sound off
slientMode = False # Set to true if you want to turn off the buzzer

playerName = "No name given" # Define the defult playerName
playerScore = 0 # Player gets a point for every correct button press
leaderboardFile = "leaderboard.txt"

melodies = [
    # Win melody
    [(660,0.1),(660,0.1),(0,0.1),(660,0.1),(0,0.1),(510,0.1),(660,0.1),(770,0.1),(0,0.1),(380,0.1),
    (0,0.1),(510,0.1),(0,0.1),(380,0.1),(320,0.1),(0,0.1)],
    
    # Lose melody
    [(220,0.3),(140,0.3),(180,0.3),(100,0.5),(80,0.5)],
    
    # Start game
    [(660, 0.2), (0, 0.05),  # "3"
    (740, 0.2), (0, 0.05),  # "2"
    (830, 0.2), (0, 0.05),  # "1"
    (880, 0.1), (988, 0.1), (1047, 0.1),  # Quick build-up
    (1320, 0.3)],
    
    # Menu music
    [(220, 0.1), (440, 0.1), (330, 0.1), (220, 0.1), (0, 0.05), (220, 0.1), (440, 0.1), (330, 0.1), (220, 0.1), (0, 0.05),  
    (330, 0.1), (440, 0.1), (554, 0.1), (440, 0.1), (330, 0.1), (220, 0.1), (0, 0.05), (220, 0.1), (330, 0.1), (440, 0.1),  

    (220, 0.1), (440, 0.1), (330, 0.1), (220, 0.1), (0, 0.05), (220, 0.1), (440, 0.1), (330, 0.1), (220, 0.1), (0, 0.05),  
    (330, 0.1), (440, 0.1), (554, 0.1), (440, 0.1), (330, 0.1), (220, 0.1), (0, 0.05), (220, 0.1), (330, 0.1), (440, 0.1),  

    (554, 0.1), (440, 0.1), (330, 0.1), (220, 0.1), (0, 0.05), (220, 0.1), (330, 0.1), (440, 0.1), (554, 0.1), (0, 0.05),
    (660, 0.15), (554, 0.15), (440, 0.15), (330, 0.15), (220, 0.2), (0, 0.2)]

]

# Level up tune
def play_level_up_tune():
    for tone in [329, 392, 659, 523, 587, 784]:
        play_tone(tone, 150)
        time.sleep(0.15)


#################################################
#           Setup function
#
# 1. Checks lights and buzzer.
# 2. Make sure we have a leaderboard file
# 3. Ask for player's name
# 4. Start the game
#################################################
def setup():
    global playerName

    freq = 1000 # Temporary variable to track current freqency of buzzer
    
    setRGBLight("off")

    if slientMode == False:
        buzzer.duty_u16(5000) # Turn on buzzer
    
    # Go through each colour of the rgb light and test each one. Also turn up the frequency of the buzzer each time.
    for light in rgbLight:
        light.on() # Turn on one of the lights
        
        buzzer.freq(freq) # Set the buzzer frequency
        sleep(0.2) # Sleep
        
        light.off() # Turn off this light
        freq += 500 # Add to frequency
    
    buzzer.freq(freq)
    setRGBLight("all") # Test all lights on at once
    sleep(0.2)
        
    buzzer.duty_u16(0) # Turn off buzzer
    setRGBLight("off")

    # Check if the leaderboard text file exists
    if leaderboardFile in os.listdir():
        print("Leaderboard file already created.")
    else:
        # Create the leaderboard file as it doesn't exist
        with open(leaderboardFile, "w") as file:
            file.write("Colour Clash Leaderboard:\n")
        print("Created the leaderboard file.")  
    
    playerName = input("Welcome to Colour Clash! Enter your player name: ") # Ask for player name
    
    # Start the game
    beginTurn()



# Helper function to set the color of the RGB light
# state can be: "all", 0, 1, 2, or "off"
def setLight(state = "all"):
    # Turn off all lights
    for light in lights:
        light.off()
    
    # Then only turn on the requested light if applicable
    if state == "all":
        for light in lights:
            light.on()
    elif state != "off":
        lights[state].on()



# Helper function to play a tone through the buzzer
def playTone(freq, duration = 0.5):
    if slientMode == False:
        buzzer.freq(freq)
        buzzer.duty_u16(30000)
        sleep(duration)
        buzzer.duty_u16(0)
    

# Helper function to play music
def playMusic(notes):
    for note in notes:
        if note[0] != 0:
            playTone(note[0], note[1])


# Helper function to add leaderboard entries
def addLeaderboardEntry(name, score):
    with open(leaderboardFile, "a") as file:
        file.write(f"{name}: {score}\n")

# Print the leaderboard
def showLeaderboard():
    with open(leaderboardFile, "r") as file:
        file.read())
    


# Game ended
def game_over(score, name):
    print("\nGame over! Your score:", score)
    play_tone(622, 300)
    time.sleep(0.3)
    play_tone(587, 300)
    time.sleep(0.3)
    play_tone(554, 300)
    time.sleep(0.3)
    for _ in range(10):
        for pitch in range(-10, 11): 
            play_tone(523 + pitch, 5) 
    time.sleep(0.5)
    save_score(name, score)
    show_leaderboard()
    

############################################################################
# Function to start a turn.
# 1. Make a random colour based on a number
# 2. Turn the RBG light on for half a second with the matching colour
# 3. Wait for one of the buttons to be pressed
# 4. When a button is pressed ->
#       Correct button pressed: Add 1 to player score, play high tone
#       Incorrect: End the game -> Record score and play sad music
#############################################################################
def beginTurn():
    global playerName
    global playerScore
    memoryArray = []
    gameStatus = ''

    playMusic(melodies[2])

    print("Let the game begin!")
    
    for i in range(15):
        colour = random.randint(0, 3) # Generates a random number from 0 - 3. Each one represents a colour
        
        setLight(colour) # Set the colour of the RGB light based on our random colour
        sleep(1)
        setLight("off")

        if gameStatus == "fail":
            break
        
        buttonPressed = False # Flag to determine whether any button has been pressed
        while buttonPressed == False:
            for button in buttons:
                if button.value() == 0:
                    if buttons.index(button) == colour:
                        playTone(2000, 0.5)
                        playerScore += 1
                        print("Correct! Current score: " + str(playerScore))

                        gameStatus = 'success'
                    else:
                        # Game over
                        playMusic(melodies[1])
                        print("Incorrect")

                        # Record score
                        addLeaderboardEntry(playerName, playerScore)

                        gameStatus = 'fail'

                    buttonPressed = True


    # Handle post game
    if gameStatus == 'success':
        addLeaderboardEntry(playerName, playerScore)
        playMusic(melodies[0])
        print("You made it " + playerName + "! You got a score of " + str(playerScore))
        showLeaderboard()


    if gameStatus == "fail":
        print("Game over " + playerName + "! You got a score of " + str(playerScore))
        showLeaderboard()
        

    # Offer for the player to start again
    if str(input("Would you like to play again? (y/n) ")) == "y":
        if str(input("Continue playing as " + playerName + "? (y/n) ")) == "y":
            beginTurn()
        else:
            setup()
    else:
        print("Hope you enjoyed the game!")


# Run the setup function
setup()

#while True:
#    playMusic(melodies[3])